# Draft of Queries

This notebook contains a mishmash of different code blocks and queries made to help in the preparation of UAI 2023. The following code was built using v1.33 and v.1.34 of OpenReview.

https://github.com/openreview/openreview-py/tags

## Initialisation
Make sure to have your credentials stored in `../credentials.py` or adjust the code accordingly. The format should be

```
{"username": "user@institution.ac.uk", "password": "asdfasdfasdf"}
```

In [3]:
%%time
# For approved details
import json

from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
from tqdm import tqdm
import openreview


def get_country(university):
    geolocator = Nominatim(user_agent="my_app")
    try:
        location = geolocator.geocode(university, language="en", namedetails=True, addressdetails=True)
        return location.raw['address']['country']
    except:
        return 'Unknown'
    

with open(r'../credentials.py') as f:
    credentials = json.load(f)
    
with open(r'../credentials.py') as f:
    credentials = json.load(f)
    
client = openreview.Client(
    baseurl='https://api.openreview.net', 
    username=f"{credentials['username']}", 
    password=f"{credentials['password']}"
)

venue_id='auai.org/UAI/2023/Conference'
blind_submissions = client.get_all_notes(
    invitation=venue_id+"/-/Blind_Submission",
    details='directReplies'
)
unblinded_submissions = client.get_all_notes(
    invitation=venue_id+"/-/Submission",
    details='directReplies'
)

CPU times: user 338 ms, sys: 43 ms, total: 381 ms
Wall time: 3.58 s


## 1. Measuring Review Length

This was used to assess the contribution of reviewers to the event by measuring the total comment length across all their papers.

In [ ]:
results_list = []
comments_list = []
author_comment_list = []
for submission in tqdm(blind_submissions):
    title = submission.content['title']
    number = submission.number
    authors = submission.content['authors']
    authorids = submission.content['authorids']
    paper_decision = submission.details['directReplies'][-1]['content']['decision']
    
    direct_replies = submission.details['directReplies']
    
    author_response_count = 0
    author_response_length = 0
    for reply in direct_replies:
        if reply['invitation'].endswith("Official_Review"):
            reviewer = client.get_groups(reply['signatures'])[0].members[0]
            str_comments = (
                reply['content']['Q3 Main strengths'] + 
                reply['content']['Q4 Main weakness'] +
                reply['content']['Q5 Detailed comments to the authors']
            )
            length_comments = len(str_comments)
            reviewer_paper_rating = int(reply['content']['Q6 Overall score'][0])
            reviewer_accept_or_reject = reviewer_paper_rating > 4
            
            results_list.append([
                  title, number, paper_decision, reviewer, length_comments, reviewer_paper_rating, reviewer_accept_or_reject
            ])
        if reply['invitation'].endswith("Official_Comment"):
            if any(['Reviewer' in x_i for x_i in reply['signatures']]):
                reviewer = client.get_groups(reply['signatures'])[0].members[0]
                comments_list.append([
                      title, number, reviewer, 1
                ])
            if any(['Authors' in x_i for x_i in reply['signatures']]):
                author_response_count += 1
                author_response_length += len(reply['content']['comment'])
    author_comment_list.append([
          number, author_response_count, author_response_length
    ])

In [ ]:
reviews_df = pd.DataFrame(results_list, columns=['title', 'paper_number', 'meta_reviewer_decision', 'reviewer', 'length_comments', 'reviewer_rating', 'reviewer_accept_or_reject'])
comments_df = pd.DataFrame(comments_list, columns=['title', 'paper_number', 'reviewer', 'number_of_comments'])
author_comment_df = pd.DataFrame(author_comment_list, columns=['paper_number', 'num_author_comments', 'length_author_comments']).drop_duplicates()
comments_df = comments_df.groupby(['title', 'paper_number', 'reviewer'])[['number_of_comments']].sum().reset_index()
results_total_df = pd.merge(reviews_df, comments_df, on=['title', 'paper_number', 'reviewer'], how='left')
results_total_df = pd.merge(results_total_df, author_comment_df, on=['paper_number'], how='left').fillna(0)
results_total_df.to_csv('review_lengths.csv', index=False)

In [ ]:
results_total_df.to_csv('review_lengths.csv', index=False)

## 2. Details of Conference Participants

### 2a. Details of Accepted Paper Authors (Plus MSFT affiliation)

This code chunk gets the author list for all accepted papers, finds their instituion, and estimates the country of their institution (using the `get_country` function).

It also does a hacky way of flagging Microsoft affiliation of authors (checking that they have an affiliation with MSFT that has not ended). This can be modified throught changing the function `find_microsoft_affiliation`.

In [17]:
%%time
# For approved details
import json

from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
from tqdm import tqdm
import openreview


def get_country(university):
    geolocator = Nominatim(user_agent="my_app")
    try:
        location = geolocator.geocode(university, language="en", namedetails=True, addressdetails=True)
        return location.raw['address']['country']
    except:
        return 'Unknown'
    
def find_microsoft_affiliation(hist):
    current_msft_affiliation = False
    for position in hist:
        if (position['end'] == None) and ('microsoft' in position['institution']['name'].lower()):
            current_msft_affiliation = True
    return current_msft_affiliation
    
with open(r'../credentials.py') as f:
    credentials = json.load(f)
    
client = openreview.Client(
    baseurl='https://api.openreview.net', 
    username=f"{credentials['username']}", 
    password=f"{credentials['password']}"
)

venue_id='auai.org/UAI/2023/Conference'
blind_submissions = client.get_all_notes(
    invitation=venue_id+"/-/Blind_Submission",
    details='directReplies'
)
unblinded_submissions = client.get_all_notes(
    invitation=venue_id+"/-/Submission",
    details='directReplies'
)

CPU times: user 301 ms, sys: 39.1 ms, total: 340 ms
Wall time: 3.3 s


In [ ]:
%%time
unblinded_details = []
for submission in tqdm(unblinded_submissions):
    title = submission.content['title']
    number = submission.number
    authors = submission.content['authors']
    authorids = submission.content['authorids']
                    
    for i, authorid in enumerate(authorids):
        try:
            author_email = openreview.tools.get_profiles(client, [f'{authorid}'])[0].content['preferredEmail']
            author_hist = client.get_profile(authorid).content['history']
            current_institution = author_hist[0]['institution']['name']
            current_msft_affiliation = find_microsoft_affiliation(author_hist)
            unblinded_details.append([
                title, number, authors[i], author_email, current_institution, current_msft_affiliation
            ])
        except:
            author_email = 'NaN'
            current_institution = 'NaN'
            unblinded_details.append([
                title, number, authors[i], author_email, current_institution
            ])

blinded_results = []                               
for submission in tqdm(blind_submissions):
    number = submission.number
    print(submission)
    decision = submission.details['directReplies'][-1]['content']['decision']
    blinded_results.append([
        number, decision
    ])

In [ ]:
unblinded_details_df = pd.DataFrame(unblinded_details, columns=['title', 'paper_id', 'author', 'email', 'institution', 'current_msft_affiliation'])
blinded_details_df = pd.DataFrame(blinded_results, columns=['paper_id', 'decision'])
full_results = pd.merge(unblinded_details_df, blinded_details_df, on='paper_id')
full_results_filtered = full_results.loc[full_results['decision'] != 'Reject']
# full_results_filtered['contains_microsoft'] = full_results_filtered['institution'].str.lower().str.contains('microsoft')
full_results_filtered.to_csv('msft_flag.csv')

In [ ]:
full_results_filtered.groupby('paper_id')['current_msft_affiliation'].agg(any).value_counts()

### 2b. Details of Area Chairs and Reviewers

In [27]:
%%time
reviewers = client.get_group(venue_id+'/Reviewers')
area_chairs = client.get_group(venue_id+'/Area_Chairs')

details = []
for ac in area_chairs.members:
    try:
        ac_hist = client.get_profile(ac).content['history']
        current_institution = ac_hist[0]['institution']['name']
        current_msft_affiliation = find_microsoft_affiliation(ac_hist)
        details.append(['AC', ac, current_institution, current_msft_affiliation])
    except:
        pass
    
for reviewer in reviewers.members:
    try:
        reviewer_hist = client.get_profile(reviewer).content['history']
        current_institution = reviewer_hist[0]['institution']['name']
        current_msft_affiliation = find_microsoft_affiliation(reviewer_hist)
        details.append(['Reviewer', reviewer, current_institution, current_msft_affiliation])
    except:
        pass

details_df = pd.DataFrame(details, columns=['Role', 'ID', 'Current_Institution', 'Current_MSFT_Affiliation'])

CPU times: user 3.39 s, sys: 519 ms, total: 3.91 s
Wall time: 2min 11s


In [29]:
details_df['Role'].value_counts()

Role
Reviewer    785
AC           79
Name: count, dtype: int64

In [30]:
details_df.groupby('Role')['Current_MSFT_Affiliation'].agg(sum)

Role
AC          2
Reviewer    9
Name: Current_MSFT_Affiliation, dtype: int64

## 3. Obtaining Review Scores

This chunk contains a hacky way of extracting the overall review scores for all papers.

In [ ]:
%%time
import json

from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
from tqdm import tqdm
import openreview


def get_country(university):
    geolocator = Nominatim(user_agent="my_app")
    try:
        location = geolocator.geocode(university, language="en", namedetails=True, addressdetails=True)
        return location.raw['address']['country']
    except:
        return 'Unknown'
    

with open(r'../credentials.py') as f:
    credentials = json.load(f)
    
client = openreview.Client(
    baseurl='https://api.openreview.net', 
    username=f"{credentials['username']}", 
    password=f"{credentials['password']}"
)

venue_id='auai.org/UAI/2023/Conference'
blind_submissions = client.get_all_notes(
    invitation=venue_id+"/-/Blind_Submission",
    details='directReplies'
)
unblinded_submissions = client.get_all_notes(
    invitation=venue_id+"/-/Submission",
    details='directReplies'
)

institutions = [] 
unblinded_pid_list = []

for submission in tqdm(unblinded_submissions):
    first_author_id = submission.content['authorids'][0]
    try:
        profile = client.get_profile(first_author_id)
        institutions.append(profile.content['history'][0]['institution']['name'])
        unblinded_pid_list.append(submission.number)
    except:
        print(f"Profile {first_author_id} not found.")

review_count = []
blinded_pid_list = []
review_scores_list = []
for submission in tqdm(blind_submissions):
    try:
        blinded_pid_list.append(submission.number)
        reviewer_scores = []
        for reply in submission.details["directReplies"]:
            if reply["invitation"].endswith("Official_Review"):
                if 'content' in reply and 'Q6 Overall score' in reply['content']:
                    s = int(reply['content']['Q6 Overall score'].split(':')[0])
                    reviewer_scores.append(s)
        review_scores_list.append(np.mean(np.array(reviewer_scores)))
        review_count.append(len(reviewer_scores))
        
    except:
        print(f"Profile {first_author_id} not found.")
        
institutions_df = pd.DataFrame({
    'institution': institutions,
    'pid': unblinded_pid_list,
})
scores_df = pd.DataFrame({
    'pid': blinded_pid_list,
    'scores': review_scores_list,
    'count': review_count
})
full_df = institutions_df.merge(
    scores_df, 
    on='pid', 
    how='inner'
).sort_values('scores', ascending=False)

# Reducing size for computational ease
full_df = full_df.iloc[1:250]

institution_country_map = pd.DataFrame({'institution': full_df['institution'].to_list()}).drop_duplicates()
institution_country_map['country'] = institution_country_map['institution'].apply(get_country)
full_df = full_df.merge(institution_country_map, on='institution')
full_df.loc[1:200, ['institution', 'scores', 'country']].to_csv('institution_counts.csv', index=False)

In [ ]:
full_df.loc[1:200, ['institution', 'count', 'country']].groupby(['institution', 'country']).count().to_csv('top_responses.csv')
full_df.loc[1:200, ['institution', 'count', 'country']].groupby('country').count()[['count']].sort_values(
    by='count', ascending=False
).to_csv('aggregated_institution_counts.csv')

## 4. Sending Emails

This chunk contains a framework for sending papers to a subset of candidates (e.g. for when you need to target specific papers rather than just accept/reject, but when the set of papers is too large to be clicking through the OR UI!).

In [ ]:
import json
import re

import pandas as pd
from tqdm import tqdm
import openreview

poster_papers = pd.read_csv('posters.csv')

with open(r'../credentials.py') as f:
    credentials = json.load(f)
    
client = openreview.Client(
    baseurl='https://api.openreview.net', 
    username=f"{credentials['username']}", 
    password=f"{credentials['password']}"
)

venue_id='auai.org/UAI/2023/Conference'

blind_submissions = client.get_all_notes(
    invitation=venue_id+"/-/Blind_Submission",
    details='replies'
)
submissions = client.get_all_notes(
    invitation=venue_id+"/-/Submission",
    # details='replies'
)

In [ ]:
spotlight_papers = [
	91,190,196,227,303,325,472,617,
	651,707,804,56,654,186,643,297,480,
	79,257,294,354,476,516,556,559,
	41,261,747,658,611,797,764
]

The command to send an email is shown below:

In [ ]:
client.post_message(
    subject='[UAI 2023] Spotlight Presentation Uploads',
    recipients=['testemail@gmail.com'],
    message=message
)

One can create a template for the email, as a function with string args. These can be used to customise the email according to the recipient's details.

In [ ]:
def email_str_template(fullname, number, title):
    return f"""
Dear Discussants,

As we get closer to the start of UAI, we’re looking forward to hearing about
the exciting new research in our community! To facilitate easy transition of
speakers during the sessions, please upload your discussions as a PDF using 
the Google form linked below by July 28, AOE.

https://forms.gle/wh1pbXwNRyTrjL2C9

Reminder that discussions are short two minute presentations that are meant 
to provide brief commentary, constructive criticism, or highlight key takeaways,
so please have no more than two slides, excluding the title slide.

We look forward to seeing you next week!

All the best,

Robin Evans and Ilya Shpitser
UAI 2023 Program Chairs
uai2023programchairs@gmail.com
"""

paper_numbers = spotlight_papers

i = 0
j = 0
for sub in tqdm(submissions):
    content = sub.content
    number = sub.number
    title = content['title']
    authorids = content['authorids']
    authornames = content['authors']
    j += 1
    # if number in poster_papers_df['number'].values:
    # if number in paper_numbers:
    #     for name, author_id in zip(authornames, authorids):
    #         message = email_str_template(name, number, title)
    #         # client.post_message(
    #         #     subject='[UAI 2023] Reminder to submit Camera-Ready including source files',
    #         #     recipients=[author_id],
    #         #     message=message
    #         # )
    #         print(message)
    #         i += 1
    if number in paper_numbers:
        print(authorids, number)
        message = email_str_template('', number, title)
        # client.post_message(
        #     subject='[UAI 2023] Spotlight Presentation Uploads',
        #     recipients=authorids,
        #     message=message
        # )
        print(message)
        i += 1            

In [ ]:
discussant_emails = [
    '...'
]

def email_str_template(fullname, number, title):
    return f"""
Dear Discussants,

As we get closer to the start of UAI, we’re looking forward to hearing about
the exciting new research in our community! To facilitate easy transition of
speakers during the sessions, please upload your discussions as a PDF using 
the Google form linked below by July 28, AOE.

https://forms.gle/wh1pbXwNRyTrjL2C9

Reminder that discussions are short two minute presentations that are meant 
to provide brief commentary, constructive criticism, or highlight key takeaways,
so please have no more than two slides, excluding the title slide.

We look forward to seeing you next week!

All the best,

Robin Evans and Ilya Shpitser
UAI 2023 Program Chairs
uai2023programchairs@gmail.com
"""

message = email_str_template('', number, title)
client.post_message(
    subject='[UAI 2023] Discussant Slides Uploads',
    # recipients=['testemail@gmail.com'],
    recipients=discussant_emails,
    message=message
)

In [ ]:
message = email_str_template('', '1234', 'asdf')

client.post_message(
    subject='[UAI 2023] Slides Deadline and Recording',
    recipients=['testemail@gmail.com'],
    message=message
)

In [ ]:
# target_papers = pd.read_csv('oral_papers.csv')#[['number', 'ac recommendation']]

In [ ]:
# paper_numbers = target_papers['number'].values

In [ ]:
paper_numbers = [
1,2,3,4,5,6
]

In [ ]:
# poster_papers_df = pd.read_csv('accpt_noCRC_2023-06-08_1330.csv')

# submissions = client.get_all_notes(
#     invitation=venue_id+"/-/Submission",
#     # details='replies'
# )

In [ ]:
i = 0
j = 0
for sub in tqdm(submissions):
    content = sub.content
    number = sub.number
    title = content['title']
    authorids = content['authorids']
    authornames = content['authors']
    j += 1
    # if number in poster_papers_df['number'].values:
    # if number in paper_numbers:
    #     for name, author_id in zip(authornames, authorids):
    #         message = email_str_template(name, number, title)
    #         # client.post_message(
    #         #     subject='[UAI 2023] Reminder to submit Camera-Ready including source files',
    #         #     recipients=[author_id],
    #         #     message=message
    #         # )
    #         print(message)
    #         i += 1
    if number in paper_numbers:
        print(authorids, number)
        message = email_str_template('', number, title)
        client.post_message(
            subject='[UAI 2023] No authors presenting',
            recipients=authorids,
            message=message
        )
        print(message)
        i += 1            

In [ ]:
submissions[437]

## 5. Checking Camera Ready Submissions

This chunk contains a very *very* hacky way of checking the camera-ready status of submissions and cross compare it against the number of revision invitations made.

In [ ]:
import json
import re

import pandas as pd
from tqdm import tqdm
import openreview

with open(r'../credentials.py') as f:
    credentials = json.load(f)
    
client = openreview.Client(
    baseurl='https://api.openreview.net', 
    username=f"{credentials['username']}", 
    password=f"{credentials['password']}"
)

venue_id='auai.org/UAI/2023/Conference'

In [ ]:
revision_invitations = list(openreview.tools.iterget_invitations(client, super = f"{venue_id}/-/Camera-Ready_Copies", expired=True))

In [ ]:
submissions_by_number = {p.number: p for p in client.get_all_notes(invitation = f'{venue_id}/-/Submission')}

In [ ]:
revisions_by_forum = {}
for invitation in revision_invitations: 
    number = int((invitation.id.split('/-/')[0]).split('Paper')[1])
    submission = submissions_by_number[number]
    references = client.get_references(referent = submission.id, invitation = invitation.id)
    if references:
        revisions_by_forum[submission.forum] = references
    else:
        pass
        # print(f'no revisions for {number}')
print(f'Number of revision invitations: {len(revision_invitations)}')
print(f'Number of revisions submitted: {len(revisions_by_forum.keys())}')

In [ ]:
submissions_by_number[number].content

In [ ]:
revisions_by_forum = {}

not_submitted = {}
results = []
for invitation in revision_invitations: 
    number = int((invitation.id.split('/-/')[0]).split('Paper')[1])
    submission = submissions_by_number[number]
    references = client.get_references(referent = submission.id, invitation = invitation.id)
    if references:
        cont = submissions_by_number[number].content
        title = cont['title']
        authorids = '|'.join(cont['authorids'])
        not_submitted = False
        submitted_sup_material = cont.get('supplementary_material', False)
        submitted_other_sup_material = cont.get('other_supplementary_material', False)
    else:
        cont = submissions_by_number[number].content
        title = cont['title']
        authorids = '|'.join(cont['authorids'])
        not_submitted = True
        submitted_sup_material = cont.get('supplementary_material', False)
        submitted_other_sup_material = cont.get('other_supplementary_material', False)
    if not_submitted:
        results.append([number, title, authorids, not_submitted, submitted_sup_material, submitted_other_sup_material])
    elif not all([submitted_sup_material, submitted_other_sup_material]):
        results.append([number, title, authorids, not_submitted, submitted_sup_material, submitted_other_sup_material])
results_df = pd.DataFrame(results, columns=['number', 'title', 'authorids', 'not_submitted', 'submitted_supplementary', 'submitted_other_supplementary'])
        # print(f'no revisions for {number}')
    
results_df.to_csv('not_submitted_list.csv', index=False)
print(f'Number of revision invitations: {len(revision_invitations)}')
print(f'Number of revisions submitted: {len(revisions_by_forum.keys())}')

In [ ]:
results_df.sort_values(by='number')

In [ ]:
_zip = client.get_attachment(id=id, field_name='other_supplementary_material')

In [ ]:
id = submissions_by_number[1].id
_zip = client.get_attachment(id=id, field_name='other_supplementary_material')

In [ ]:
from pathlib import Path
outdir = Path(".")
not_submit = [1234]

submissions = openreview.tools.iterget_notes(
        client, invitation=venue_id)

for l in submissions:
    if l.number in not_submit:
        print('\n')
        print(f'{l.number}: ', end="")
        print(l)
        number = l.number
        id = l.id
        paper_path = outdir / f'{number}'/ 'submitted'
        paper_path.mkdir(parents=True, exist_ok=True)
        # latex zip
        # check if it is correctly named as 'author_number/latex/'
        if 'other_supplementary_material' in l.content:
            _zip = client.get_attachment(id=id, field_name='other_supplementary_material')
            with open(paper_path / f'{number}.zip','wb') as op:
                op.write(_zip)
        # author submitted pdf and supp pdf
        with open(paper_path / f'{number}.pdf','wb') as op: 
            op.write(client.get_attachment(id=id, field_name='pdf'))
        if 'supplementary_material' in l.content and l.content['supplementary_material']:
            with open(paper_path / f'{number}-supp.pdf','wb') as op: 
                op.write(client.get_attachment(id=id, field_name='supplementary_material'))

In [ ]:
submissions_by_number[91].content

## 6. List of authors, papers and their affiliations

Simple way of checking the affiliations of the authors who submitted to the conf.

In [ ]:
blind_submissions = client.get_all_notes(
    invitation=venue_id+"/-/Blind_Submission",
    details='replies'
)
submissions = client.get_all_notes(
    invitation=venue_id+"/-/Submissions",
    details='replies'
)

In [ ]:
paper_outputs = []
for b_sub in blind_submissions:
    decision = b_sub.details['replies'][-1]['content']['decision']
    if decision == 'Accept':
        paper_outputs.append([
            b_sub.content['title'],
            b_sub.number
        ])
paper_outputs_df = pd.DataFrame(paper_outputs, columns=['title', 'number'])
        
author_paper_map = []
for sub in submissions:
    number = sub.number
    for id_, author in zip(sub.content['authorids'], sub.content['authors']):
        author_paper_map.append([
            number, id_, author
        ])
author_paper_map_df = pd.DataFrame(author_paper_map, columns=['number', 'id', 'author'])

partial_results_df = pd.merge(paper_outputs_df, author_paper_map_df, on='number', how='inner')

In [ ]:
author_results = openreview.tools.get_profiles(client, list(partial_results_df['id']))
author_institution_map = []
for result in author_results:
    id_ = result.id
    try:
        institution = result.content['history'][0]['institution']['name']
    except:
        instituion = 'NA'
    author_institution_map.append([
          id_, institution
    ])
author_institution_map_df = pd.DataFrame(author_institution_map, columns=['id', 'institution'])

full_results_df = pd.merge(partial_results_df, author_institution_map_df, on='id', how='left')

In [ ]:
full_results_df

In [ ]:
full_results_df.to_csv('author_institution_map.csv', index=False)

# 7. Change Permissions for accepted authors not to have their reviews posted publicly

Was a bit of an onerous job to do this -- need to go through all the reviews, find the ID's associated with the comments, and hide them.

In [ ]:
import json
import re

import pandas as pd
from tqdm import tqdm
import openreview

with open(r'../credentials.py') as f:
    credentials = json.load(f)
    
client = openreview.Client(
    baseurl='https://api.openreview.net', 
    username=f"{credentials['username']}", 
    password=f"{credentials['password']}"
)

venue_id='auai.org/UAI/2023/Conference'
blind_submissions = client.get_all_notes(
    invitation=venue_id+"/-/Blind_Submission",
    details='replies'
)
submissions = client.get_all_notes(
    invitation=venue_id+"/-/Submission",
    # details='replies'
)

ids = {
    1234: [

        'asdf', 
        'qewr', 
        'wewrt', 
        'sdfg'
    ],
    234: [
        'asdf-M7',
        'qewr',
        'erty'
    ]
}

def private_reader_notes(number):
    return [
        f"auai.org/UAI/2023/Conference/Program_Chairs", 
        f"auai.org/UAI/2023/Conference/Paper{number}/Area_Chairs", 
        f"auai.org/UAI/2023/Conference/Paper{number}/Reviewers/Submitted", 
        f"auai.org/UAI/2023/Conference/Paper{number}/Authors"
    ]


        
for paper_num, review_ids in ids.items():
    for id_ in review_ids:
        print(paper_num, id_)
        note = client.get_note(id_)
        print(note.readers)
        note.readers = private_reader_notes(paper_num)
        print(note.readers)

        # client.post_note(note)

In [ ]:
blind_submissions[0].id
blind_submissions[0].number

In [ ]:
papers_to_change = [1234, 2345]

In [ ]:
def private_reader_notes(number):
    return [
        f"auai.org/UAI/2023/Conference/Program_Chairs", 
        f"auai.org/UAI/2023/Conference/Paper{number}/Area_Chairs", 
        f"auai.org/UAI/2023/Conference/Paper{number}/Reviewers/Submitted", 
        f"auai.org/UAI/2023/Conference/Paper{number}/Authors"
    ]

In [ ]:
ids = []
for sub in blind_submissions:
    if sub.number in papers_to_change:
        ids.append(sub.id)

In [ ]:
note = client.get_note(ids[211][0])
note.readers
note.number

In [ ]:
# note = client.get_note('2iO6lcKXkF')
note.readers = private_reader_notes(211)
print(note.invitation)
print(note.content['title'])

In [ ]:
note.readers

In [ ]:
client.post_note(note)

In [ ]:
ids = []
for sub in blind_submissions:
    if sub.number in papers_to_change:
        note = client.get_note(sub.id)
        note.readers = private_reader_notes(sub.number)
        client.post_note(note)